# osTicket

Con esta guía de Jupyter vamos a lograr instalar una versión reciente de osTicket, en un servidor Ubuntu 16.04 recién instalado.

Primero debe instalar los componentes del servidor LAMP, inciciando por la instalación de la base de datos. Seguidamente descargamos el código de osTicket.

Por último, creamos la base de datos a ser utilizada por osTicket, y luego corremos el instalador Web.


### MariaDB
MySQL y MariaDB tienen algunas pequeñas diferencias pero para nuestro objetivo son técnicamente equivalente, por lo cual vamos a preferir MariaDB sobre MySQL.

A la hora de instalar la base de datos de MariaDB, la herramienta de Apt de Debian va a insistir en solicitarnos una contraseña para el usuario "root" de la base de datos, el cual vamos a utilizar para tareas de administración.

Para evitar que en esta guía se nos solicite contraseña, vamos a usar el comando ``debconf-set-selections`` para definir la contraseña de antemano.

Nota: El usuario "root" de MariaDB y el de Linux son completamente aparte.

In [ ]:
echo "mariadb-server mysql-server/root_password password greencore" | sudo debconf-set-selections
echo "mariadb-server mysql-server/root_password_again password greencore" | sudo debconf-set-selections
sudo apt-get -y install mariadb-server

### Apache y Php

Procedemos ahora a instalar Apache2, con el componente de Php el cual fuerza el servidor a usar el MPM **prefork**.

Así mismo instalamos módulos de Php requeridos por la aplicación osTicket, según la documentación del fabricante.

Nota: No vamos a reiniciar el servicio de Apache en este momento, sino hasta luego de editar archivos de configuración.

In [ ]:
sudo apt-get -y install php libapache2-mod-php apache2 php7.0-cli php7.0-mysql php7.0-cgi php7.0-fpm php7.0-gd php7.0-imap php7.0-xml php7.0-mbstring php7.0-intl php-apcu
sudo a2enmod rewrite

### Cambios menores en Apache

**Desde una terminal**, y utilizando su editor preferido, debe cambiar dos archivos de la siguiente forma:

**Archivo:** ``/etc/apache2/mods-enabled/dir.conf``

**Cambio:** La línea que inicia por DirectoryIndex tiene una lista de las extensiones de archivos de tipo "Index". Ya sea cambiando el orden, o eliminando extensiones, debe asegurarse que ``index.php`` esté de primero en la lista.

``DirectoryIndex index.php``

**Archivo:** ``/etc/apache2/apache2.conf``

**Cambio:** En la definición del directorio ``/var/www`` originalmente tiene la instrucción ``AllowOverride None``. Cambie la definición para que diga ``AllowOverride All``

``<Directory /var/www/>
        Options Indexes FollowSymLinks
        AllowOverride All
        Require all granted
</Directory>``

Reiniciamos Apache y mostramos el estado del servicio.

## osTicket

La última versión y recomendada de osTicket es la v1.10

In [ ]:
sudo systemctl restart apache2
sudo systemctl status apache2 | cat
wget -qc http://osticket.com/sites/default/files/download/osTicket-v1.10.zip
ls -lh

Descomprimimos el código de osTicket dentro de /var/www/html que es el directorio default de Apache.

En un sistema en producción se recomienda crear un directorio para cada virtualhost, o un único virtualhost para osTicket y hacer uso de la función de "Multi tenant" para trabajar varios clientes con una sola instancia de la aplicación.

Importante que el código queda con derechos de **www-data**, usuario que utiliza Debian para el servicio de web de Apache. Esto porque osTicket debe poder manipular archivos dentro de la carpeta.

In [ ]:
sudo apt-get -y install unzip
sudo unzip -q ~/osTicket-v1.10.zip
sudo rm -r /var/www/html
sudo mv upload /var/www/html
sudo chown -R www-data:www-data /var/www/html
ls /var/www/html

### Creación de base de datos

Antes de ejecutar el instalador Web de osTicket debemos crear la base de datos y el usuario que usará la aplicación de osTicket.

Para ello vamos a utilizar el comando de mysql(1), definiendo la contraseña "greencore" en la misma línea de comando.

In [ ]:
sudo mysql -u root -pgreencore << EOF
CREATE DATABASE osticket;
CREATE USER osticket@localhost IDENTIFIED BY 'greencore';
GRANT ALL PRIVILEGES ON osticket.* TO osticket@localhost IDENTIFIED BY 'greencore';
FLUSH PRIVILEGES;
EOF

### Archivo de configuración de osTicket

Para poder ejecutar el instalador, debemos crear un archivo de configuración el cual debe tener **temporalmente** permisos de escritura.

Una vez completado el proceso de instalación, debemos remover los permisos de escritura del archivo. Ver al final de esta guía.

In [ ]:
sudo cp -v /var/www/html/include/ost-sampleconfig.php /var/www/html/include/ost-config.php
sudo chmod 0666 /var/www/html/include/ost-config.php
sudo chown www-data:www-data /var/www/html/include/ost-config.php

### Instalador web

No es necesario manipular ningún archivo Php para definir la configuración de nuestro sitio. Visitamos el sitio web, donde nos recibe el instalador Web de osTicket. Este funciona en estilo "wizard" donde simplemente vamos contestando con cuidado las consultas que nos hace.

http://10.42.25.7X/setup/install.php



En el instalador puede usar valores que usaría en un sitio en producción.
Recuerde utilizar para este laboratorio el usuario "admin", con contraseña "greencore" para facilitar la revisión de errores o dudas.

### Removiendo permisos

Eliminamos el permiso de escritura del archivo de configuración, así como removemos el directorio ``setup`` del código de osTicket.

In [ ]:
sudo chmod 0644 /var/www/html/include/ost-config.php
sudo rm -r /var/www/html/setup